In [ ]:
"""
Created on Sun Jun 27 01:35:11 2021

@author: MHinojosaLee
"""

In [3]:
import numpy as np
import pandas as pd
from matplotlib import animation as ani, pyplot as plt
import seaborn as sns #pretty graphics R style

from IPython.display import HTML
plt.style.use('seaborn-darkgrid')
from sklearn.inspection import permutation_importance
import matplotlib as mpl 
import matplotlib.pyplot as plt #graphics
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler #library for the rescaling
import statsmodels.api as sm 
import statsmodels.formula.api as smf
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# from treeinterpreter import treeinterpreter as ti, utils
import joblib
import getpass
from pathlib import Path

In [4]:
if getpass.getuser() == 'daniel':
    project_root_path = Path("/home/daniel/PhD/PhD classes/Python Machine learning/final-project")
    data_raw_path = project_root_path / 'data' / 'raw'
    # Lets read the trining dataset
    data_train = pd.read_csv(data_raw_path / 'train_V2.csv')
    # Now we read the training data set
    score = pd.read_csv(data_raw_path /  'score.csv')
    # We read the dictonary
    dict_features = pd.read_csv(data_raw_path /  'dictionary.csv', delimiter=';', header=None)
elif getpass.getuser() == 'maart':
    data_file_path = "C:/Users/maart/Machine Learning/ML-in-Python/data/raw/"
    # Lets read the trining dataset
    data_train = pd.read_csv(data_file_path+'train_V2.csv')
    # Now we read the training data set
    score = pd.read_csv(data_file_path+'score.csv')
    dict_features_path = data_file_path+'dictionary.csv'
    dict_features = pd.read_csv(dict_features_path, delimiter=';', header=None)
else:
      data_file_path = "C:/Users/mhinojosalee/Downloads/Machine learning with Python\Exam/"
      # Lets read the trining dataset
      data_train = pd.read_csv(data_file_path+'train_V2.csv')
      # Now we read the training data set
      score = pd.read_csv(data_file_path+'score.csv')

In [5]:
print(data_train.shape)
print(data_train.head())
pd.options.display.max_columns = None
print(data_train.describe())

(5000, 53)
   income_am  profit_last_am  profit_am  damage_am  damage_inc  crd_lim_rec  \
0      227.0             0.0     3201.0      888.0         6.0      15000.0   
1      268.0            16.0     1682.0        0.0         0.0        750.0   
2      283.0            23.0     1673.0        0.0         0.0        750.0   
3      227.0             0.0     1685.0        0.0         0.0          0.0   
4     4091.0          1028.0     3425.0      785.0         2.0      14000.0   

   credit_use_ic  gluten_ic  lactose_ic  insurance_ic  ...  score2_neg  \
0            0.0        0.0         0.0           0.0  ...         NaN   
1            0.0        0.0         0.0           1.0  ...         NaN   
2            0.0        0.0         0.0           1.0  ...    0.099529   
3            0.0        0.0         0.0           0.0  ...         NaN   
4            0.0        0.0         1.0           0.0  ...         NaN   

   score3_pos  score3_neg  score4_pos  score4_neg  score5_pos  score5

In [ ]:
# total = data_train.isnull().sum().sort_values(ascending=False)
# percent = (data_train.isnull().sum()/data_train.isnull().count()).sort_values(ascending=False)
# (data_train.isnull().sum(axis=1))[data_train.isnull().sum(axis=1) > 30]
# table
# missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
# print(missing_data.head(30))
# print(data_train['score2_pos'].value_counts())
# data_train = data_train.drop((missing_data[missing_data['Percent'] > 0.30]).index,1)
# data_train.dropna(inplace=True) 
#we could drop all that is NaN, but we will loose observations. (4425, 43) instead of (4425, 43) 

In [6]:
print(data_train.columns)
data_feat = data_train.drop(['outcome_profit', 'outcome_damage_inc', 'outcome_damage_amount'], axis=1)
print(data_feat.shape)

Index(['income_am', 'profit_last_am', 'profit_am', 'damage_am', 'damage_inc',
       'crd_lim_rec', 'credit_use_ic', 'gluten_ic', 'lactose_ic',
       'insurance_ic', 'spa_ic', 'empl_ic', 'cab_requests', 'married_cd',
       'bar_no', 'sport_ic', 'neighbor_income', 'age', 'marketing_permit',
       'urban_ic', 'dining_ic', 'presidential', 'client_segment', 'sect_empl',
       'prev_stay', 'prev_all_in_stay', 'divorce', 'fam_adult_size',
       'children_no', 'tenure_mts', 'tenure_yrs', 'company_ic', 'claims_no',
       'claims_am', 'nights_booked', 'gender', 'shop_am', 'shop_use',
       'retired', 'gold_status', 'score1_pos', 'score1_neg', 'score2_pos',
       'score2_neg', 'score3_pos', 'score3_neg', 'score4_pos', 'score4_neg',
       'score5_pos', 'score5_neg', 'outcome_profit', 'outcome_damage_inc',
       'outcome_damage_amount'],
      dtype='object')
(5000, 50)


In [7]:
print(score.shape)
datafull = pd.concat([data_feat, score])
print(datafull.shape)
print(datafull['client_segment'].value_counts())
print(datafull['sect_empl'].value_counts())
print(datafull['gender'].value_counts())
print(datafull['retired'].value_counts())
print(datafull['gold_status'].value_counts())
print(datafull['prev_stay'].value_counts())
print(datafull['divorce'].value_counts())
print(datafull['married_cd'].value_counts())

(500, 50)
(5500, 50)
1.0    3712
2.0     925
0.0     352
3.0     329
4.0      87
5.0      38
Name: client_segment, dtype: int64
0.0    4820
1.0     468
6.0      78
2.0      45
4.0      29
3.0       3
Name: sect_empl, dtype: int64
M    2734
V    2709
Name: gender, dtype: int64
0.0    4456
1.0     987
Name: retired, dtype: int64
0.0    5241
1.0     202
Name: gold_status, dtype: int64
1.0    4848
0.0     595
Name: prev_stay, dtype: int64
0.0    4884
1.0     559
Name: divorce, dtype: int64
True     4460
False    1040
Name: married_cd, dtype: int64


In [8]:
# datafull['client_segment'] = pd.Categorical(datafull['client_segment'])
# datafull['sect_empl'] = pd.Categorical(datafull['sect_empl'])
# datafull['retired'] = pd.Categorical(datafull['retired'])
# datafull['gold_status'] = pd.Categorical(datafull['gold_status'])
# datafull['prev_stay'] = pd.Categorical(datafull['prev_stay'])
# datafull['divorce'] = pd.Categorical(datafull['divorce'])

impute_mode = SimpleImputer (strategy='most_frequent')
for cols in ['client_segment', "credit_use_ic", "gluten_ic", "lactose_ic","insurance_ic","marketing_permit", "presidential", "urban_ic", "prev_all_in_stay", "shop_use", 
             "company_ic", "dining_ic", "spa_ic","sport_ic","empl_ic",'sect_empl', "retired", "gold_status", "prev_stay", 'divorce', "gender"]:  
      datafull[cols] = impute_mode.fit_transform(datafull[[cols]])

In [9]:
datafull['client_segment'] = pd.Categorical(datafull['client_segment'])
datafull['sect_empl'] = pd.Categorical(datafull['sect_empl'])
# The NaN categorie won't be necessary anymore, thanks to the mode imputing.
pd.get_dummies(datafull[['client_segment', 'sect_empl']], dummy_na=False).head()
print(datafull.shape)
datafull2 = pd.concat([datafull,pd.get_dummies(datafull[['gender','client_segment', 'sect_empl']], dummy_na=False)], axis=1)
print(datafull2.shape)
print(datafull2.head(1000))     

(5500, 50)
(5500, 64)
     income_am  profit_last_am  profit_am  damage_am  damage_inc  crd_lim_rec  \
0        227.0             0.0     3201.0      888.0         6.0      15000.0   
1        268.0            16.0     1682.0        0.0         0.0        750.0   
2        283.0            23.0     1673.0        0.0         0.0        750.0   
3        227.0             0.0     1685.0        0.0         0.0          0.0   
4       4091.0          1028.0     3425.0      785.0         2.0      14000.0   
..         ...             ...        ...        ...         ...          ...   
995     3103.0             0.0     9466.0     1206.0         2.0      12500.0   
996        NaN             NaN        NaN        NaN         NaN          NaN   
997      250.0           823.0     1646.0        0.0         0.0       1500.0   
998     6382.0           561.0     7265.0        0.0         0.0       1500.0   
999     5556.0          2464.0     2464.0        0.0         0.0          0.0   

     

In [10]:
print(datafull2.shape)
datafull2.drop(['client_segment', 'sect_empl', 'gender', 'client_segment_5.0','sect_empl_6.0','gender_V'], axis=1, inplace=True)
print(datafull2.shape)

datafull2['profitpernight'] = datafull2['profit_am'] / datafull2['nights_booked']

(5500, 64)
(5500, 58)


In [11]:
# During class it was mentioned that sometimes it was not worthy to use a very complex thing like sof imputing to avoid dropping these features. 
# I am using something simple, that is a mean imputing, because these columns were the scores, and I did not want to drop them.
# The scores are quantitative.

impute_quant = SimpleImputer (strategy='mean')
for cols in ['score1_pos', 'score1_neg', 'score2_pos', 'score2_neg', 'score3_pos',
       'score3_neg', 'score4_pos', 'score4_neg', 'score5_pos', 'score5_neg']:  # Missing data, Scores are quantitative
      datafull2[cols] = impute_quant.fit_transform(datafull2[[cols]])

In [12]:
print(datafull2.shape)
datafull2.dropna(thresh = datafull2.shape[1]*0.3, axis = 0, inplace = True)
print(datafull2.shape)
# And here we find that there are not missing values from the rows (no missing values per row). So we go to imputting the rest of the missing values

(5500, 59)
(5500, 59)


In [13]:
print(datafull2.isnull().sum().sum())
datafull2.fillna(datafull2.mean(), inplace=True)
print(datafull2.isnull().sum().sum())

2033
0


In [14]:
scaler = StandardScaler()
datafull3 = pd.DataFrame(scaler.fit_transform(datafull2))
datafull3.columns = datafull2.columns

In [15]:
data_train = pd.concat([data_train[['outcome_profit', 'outcome_damage_inc', 'outcome_damage_amount']],datafull3[0:5000]], axis=1)
print(data_train.shape)
score = datafull3[5000:5500] #The score dataset will be the last 500 observations
score.shape

(5000, 62)


(500, 59)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(data_train.drop(['outcome_profit', 'outcome_damage_inc', 'outcome_damage_amount'],1),
                                                    data_train['outcome_profit'], test_size=0.2, random_state=48)

In [ ]:
# Our model will try 500 random hyperparameter combinations, each time using 5 Cross Validation folds, totalling 2500 fits
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 50)]
learning_rate = [x for x in np.logspace(start = -3, stop = -0.01, num = 50)]
max_features = ['auto']
max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]
min_samples_split = [2, 5, 10, 30]
min_samples_leaf = [1, 2, 4, 10, 30]
subsample = [0.4, 0.6, 0.8, 1]
random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'subsample': subsample}
gbm = GradientBoostingRegressor()
gbm_random = RandomizedSearchCV(estimator = gbm, param_distributions = random_grid, n_iter = 500, cv = 5, verbose=2, random_state=42, n_jobs = -1)
gbm_random.fit(X_train, y_train)
gbm_random.best_params_

In [ ]:
joblib.dump(gbm_random, 'random_search_gbm.pkl')

In [ ]:
pd.DataFrame(gbm_random.cv_results_).head()

In [ ]:
pd.DataFrame(gbm_random.cv_results_).loc[pd.DataFrame(gbm_random.cv_results_)['mean_test_score'].idxmax()]

In [ ]:
params = gbm_random.best_params_
gbm_profit = GradientBoostingRegressor(**params)
gbm_profit.fit(X_train, y_train)
# I got R2: 0.953 for the X_train and R2: 0.822 for the X_test
print('R2: %.3f' % gbm_profit.score(X_train, np.array(y_train).reshape(-1,1)))
print('R2: %.3f' % gbm_profit.score(X_test, np.array(y_test).reshape(-1,1))) #Here we are using the "holdout" set already

In [ ]:
profit_preds = gbm_profit.predict(score)

In [ ]:
# We were interested in identifying what were the variables that mattered, so we used Variable importances based on impurity reduction.

gbm_profit.feature_importances_.sum()
d = {'feature':X_train.columns, 'importance':gbm_profit.feature_importances_}
importances = pd.DataFrame(data=d)
importances.sort_values('importance', ascending=False,inplace=True)

plt.rcdefaults()
plt.rcParams['figure.figsize'] = (4, 3)
fig, ax = plt.subplots()
variables = importances.feature
y_pos = np.arange(len(variables))
scaled_importance = importances.importance
ax.barh(y_pos, scaled_importance, align='center', color='deepskyblue', ecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(variables)
ax.invert_yaxis()
ax.set_xlabel('Scaled Importance')
ax.set_title('Variable Importance')
plt.show()
# This graphic is unorganized. All the variables are shown in the y axis, it is difficult to read them. So Next, I will organize it.

In [ ]:
importances2 = importances.copy()
importances2 = importances2.head(20)
import matplotlib.pyplot as plt
plt.rcdefaults()
plt.rcParams['figure.figsize'] = (4, 3)
fig, ax = plt.subplots()
variables = importances2.feature
y_pos = np.arange(len(variables))
scaled_importance = importances2.importance
ax.barh(y_pos, scaled_importance, align='center', color='deepskyblue', ecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(variables)
ax.invert_yaxis()
ax.set_xlabel('Scaled Importance')
ax.set_title('Variable Importance')
plt.show()
# And now we have a nice plot

In [ ]:
imp = permutation_importance(gbm_profit, X_train, y_train,n_repeats=10,
                                random_state=42, n_jobs=2)



sorted_idx = imp.importances_mean.argsort()

fig, ax = plt.subplots()
ax.boxplot(imp.importances[sorted_idx][48:58].T,
           vert=False, labels=X_train.columns[sorted_idx][48:58])
ax.set_title("Permutation Importances (test set)")
fig.tight_layout()
plt.show()
# Feature importance and Permutation importance identify the same 2 strongly predictive features for our model for Profit: Profit and Nights booked, which makes sense

### Prediction of damage (binary)

In [18]:
data_train.outcome_damage_inc.value_counts()

0    3723
1    1277
Name: outcome_damage_inc, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_train.drop(['outcome_profit', 'outcome_damage_inc', 'outcome_damage_amount'],1), 
                                                    data_train['outcome_damage_inc'], test_size=0.2, random_state=9876)
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 50)]
learning_rate = [x for x in np.logspace(start = -3, stop = -0.01, num = 50)]
max_features = ['auto']
max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]
min_samples_split = [2, 5, 10, 30]
min_samples_leaf = [1, 2, 4, 10, 30]
subsample = [0.4, 0.6, 0.8, 1]
random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'subsample': subsample}
gbm = GradientBoostingClassifier()
gbm_random = RandomizedSearchCV(estimator = gbm, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=4872, n_jobs = -1)
gbm_random.fit(X_train, y_train)
gbm_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 25.3min finished


{'subsample': 0.8,
 'n_estimators': 761,
 'min_samples_split': 10,
 'min_samples_leaf': 30,
 'max_features': 'auto',
 'max_depth': 7,
 'learning_rate': 0.004690557776399856}

In [20]:
params = gbm_random.best_params_
gbm_damagebin = GradientBoostingClassifier(**params)
gbm_damagebin.fit(X_train, y_train)
print('Train accuracy: %.3f' % gbm_damagebin.score(X_train, y_train))
print('Test accuracy: %.3f' % gbm_damagebin.score(X_test, y_test))

Train accuracy: 0.829
Test accuracy: 0.730


In [21]:
damagebin_preds = gbm_damagebin.predict_proba(score)